<a href="https://colab.research.google.com/github/ctawong/Face-Depixelizer/blob/master/Face_Depixelizer_Eng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<b><font color="black" size="+4">Try PULSE</font></b>

Given a low-resolution input image, Face Depixelizer searches the outputs of a generative model (here, StyleGAN) for high-resolution images that are perceptually realistic and downscale correctly.

<b><font color="black" size="+2">Based on:</font></b>

**GitHub repository**: [PULSE](https://github.com/adamian98/pulse)

Article: [PULSE: Self-Supervised Photo Upsampling via Latent Space Exploration of Generative Models](https://arxiv.org/abs/2003.03808)





In [ ]:
import requests
url='https://dmn-dallas-news-prod.cdn.arcpublishing.com/resizer/aG7BUQ2JaxvTBCdsEi5XFKvfbTE=/1660x934/smart/filters:no_upscale()/cloudfront-us-east-1.images.arcpublishing.com/dmn/74PWSUWQWGRIUQZZQH3O4FNR6I.jpg'
r = requests.get(url, allow_redirects=True)
ext=r.headers.get('content-type').split('/')[1]
open('image_orig.'+ext, 'wb').write(r.content)

In [ ]:
!ls /content/pulse

In [ ]:
Path("PULSE.py").exists()

In [ ]:
!ls input

In [ ]:
Path("realpics").exists()

In [ ]:
import torch
import torchvision
from pathlib import Path
import shutil
import os

if not Path("PULSE.py").exists():
  if Path("pulse").exists():
    %cd /content/pulse
  else:
    !git clone https://github.com/ctawong/pulse
    %cd /content/pulse
    !mkdir input/
    toPIL = torchvision.transforms.ToPILImage()
    toTensor = torchvision.transforms.ToTensor()
    from bicubic import BicubicDownSample
    D = BicubicDownSample(factor=1)

# remove previous content
if Path("realpics").exists():
  shutil.rmtree('realpics')
!mkdir realpics
if Path("input").exists():
  shutil.rmtree('input')
!mkdir input


from io import BytesIO
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
from PULSE import PULSE
from google.colab import files
from bicubic import BicubicDownSample
from IPython import display
from IPython.display import display
from IPython.display import clear_output
import numpy as np
from drive import open_url
from mpl_toolkits.axes_grid1 import ImageGrid
import requests
%matplotlib inline

#@markdown ## Basic settings:
#@markdown ##### 
Photo_URL = "https://dmn-dallas-news-prod.cdn.arcpublishing.com/resizer/aG7BUQ2JaxvTBCdsEi5XFKvfbTE=/1660x934/smart/filters:no_upscale()/cloudfront-us-east-1.images.arcpublishing.com/dmn/74PWSUWQWGRIUQZZQH3O4FNR6I.jpg" #@param {type:"string"}
r = requests.get(Photo_URL, allow_redirects=True)
ext=r.headers.get('content-type').split('/')[1]
realPicName = 'realpics/image.'+ext
open(realPicName, 'wb').write(r.content)

!python align_face.py

face = Image.open(fn)
face = face.resize((1024, 1024), Image.ANTIALIAS)
face = face.convert('RGB')
face_name = 'face.png'
face.save(face_name)
%cp $face_name /content/pulse/input/

# images = []
# imagesHR = []
# imagesHR.append(face)
# face = toPIL(D(toTensor(face).unsqueeze(0).cuda()).cpu().detach().clamp(0,1)[0])
# images.append(face)


#@markdown ---
#@markdown ## Advanced settings:
#@markdown ##### *If you want to make a more accurate result, then modify the following* **DEFAULT** *variables*:

eps = 0.0025 #@param {type:"slider", min:0.001, max:0.005, step:0.0005}
seed = 100 #@param {type:"integer"}
noise_type = 'trainable'  #@param ['zero', 'fixed', 'trainable']
optimizer = 'adam'  #@param ['sgd', 'adam','sgdm', 'adamax']
learning_rate = 0.4 #@param {type:"slider", min:0, max:1, step:0.05}
learning_rate_schedule = 'linear1cycledrop'  #@param ['fixed', 'linear1cycle', 'linear1cycledrop']
steps = 100 #@param {type:"slider", min:100, max:1000, step:50}
clear_output()

seed = abs(seed)
print('Estimated Runtime: {}s.\n'.format(round(0.23*steps)+6))
!python run.py \
  -seed $seed \
  -noise_type $noise_type \
  -opt_name $optimizer \
  -learning_rate $learning_rate \
  -steps $steps \
  -lr_schedule $learning_rate_schedule

#@markdown ---
#@markdown *If there is an error during execution or the "**Browse**" button is not active, try running this block again*

fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(mpimg.imread('/content/pulse/input/face.png'))
ax1.set_title('Original')
ax2.imshow(mpimg.imread('/content/pulse/runs/face.png'))
ax2.set_title('Result')
plt.show()

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Download result</font></b>
try: files.download('/content/pulse/runs/face.png')
except: raise Exception("No result image")